In [9]:
import marlo
import sys
import logging
logging.basicConfig(filename='myapp.log', level=logging.INFO)
logging.basicConfig(filename='myapp.log', level=logging.ERROR)

sys.path.append("../classes/")

from MinecraftEnvironmentManager import MinecraftEnvironmentManager
from Agent import Agent
from DQN import DQN
from EpsilonGreedyStrategy import EpsilonGreedyStrategy
from QValueCalculator import QValueCalculator
from ReplayMemory import ReplayMemory
from Utils import Utils

from collections import namedtuple

import torch
import torch.optim as optim

In [2]:
Experience = namedtuple('Experience',
('state','action','next_state','reward'))

# Hyperparameters

In [3]:
batch_size = 256
gamma = 0.999
eps_start = 1
eps_end = 0.01
eps_decay = 0.001
target_update = 10
memory_size = 100000
lr = 0.001
num_episodes = 1000

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mem = MinecraftEnvironmentManager(marlo, device, 'MarLo-FindTheGoal-v0')
strategy = EpsilonGreedyStrategy(eps_start, eps_end, eps_decay)
agent = Agent(strategy, mem.num_actions_available(), device)
memory = ReplayMemory(memory_size)

INFO:marlo.base_env_builder:params.gameMode : Cannot force survival mode.
INFO:marlo.base_env_builder:params.gameMode : Cannot force survival mode.


In [13]:
mem.launch_enviroment()

Nothing is listening on port 10000 - will attempt to launch Minecraft from a new terminal.
Giving Minecraft some time to launch... 
. . . . . . . . . . . . . . . ok


In [10]:
policy_net = DQN(mem.get_screen_height(), mem.get_screen_width()).to(device)
target_net = DQN(mem.get_screen_height(), mem.get_screen_width()).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.Adam(params=policy_net.parameters(), lr=lr)

In [11]:
#what we will use to train the network
Experience = namedtuple(
'Experience',
('state','action','next_state','reward'))

In [ ]:
episode_durations = []
for episode in range(num_episodes):
    mem.reset()
    state = mem.get_state()
    
    for timestep in count():
        action = agent.select_action(state, policy_net)
        reward = mem.take_action(action)
        next_state = mem.get_state()
        memory.push(Experience(state, action, next_state, reward))
        state = next_state

        if memory.can_provide_sample(batch_size):
            experiences = memory.sample(batch_size)
            states, actions, rewards, next_states = extract_tensors(experiences)
            
            current_q_values = QValues.get_current(policy_net, states, actions)
            next_q_values = QValues.get_next(target_net, next_states)
            target_q_values = (next_q_values * gamma) + rewards

            loss = F.mse_loss(current_q_values, target_q_values.unsqueeze(1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        if mem.done:
            episode_durations.append(timestep)
            plot(episode_durations, 100)
            break

    if episode % target_update == 0:
        target_net.load_state_dict(policy_net.state_dict())
        
mem.close()